In [1]:
cd drive/MyDrive/pc_github/HandGestureRecognition/

/content/drive/MyDrive/pc_github/HandGestureRecognition


In [2]:
from os import path
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
import torch.utils.tensorboard as tb
from torch import optim
from utils import ConfusionMatrix

In [3]:
LABEL_NAMES = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
          'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del',
          'space']

In [4]:
class HGMDataset(Dataset):
  def __init__(self, csvpath):
    df = pd.read_csv(csvpath)
    df = df.sample(frac=1).reset_index(drop=True)       
    self.x = df.iloc[:,4:].values
    self.y = df['label'].values.reshape(-1,1)

  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    x_val  = torch.Tensor(self.x[idx])
    y_val  = torch.Tensor(self.y[idx])
    y_val  = y_val.type(torch.LongTensor)
    return { 'data': x_val,
            'target': y_val
            }

In [5]:
# df = pd.read_csv('asl_alphabet_train/labels.csv')
# df = df.sample(frac=1).reset_index(drop=True)       
# x = df.iloc[:,4:].values
# y = df['label'].values.reshape(-1,1)

In [6]:
HGM_data = HGMDataset('asl_alphabet_train/labels.csv')
#HGM_data.__getitem__(1)

In [7]:
## create training and validation split 
split = int(0.8 * len(HGM_data))
index_list = list(range(len(HGM_data)))
train_idx, valid_idx = index_list[:split], index_list[split:]

In [8]:
## create sampler objects using SubsetRandomSampler
tr_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)

In [9]:
## create iterator objects for train and valid datasets
trainloader = DataLoader(HGM_data, batch_size=256, sampler=tr_sampler)
validloader = DataLoader(HGM_data, batch_size=256, sampler=val_sampler)

In [10]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden1 = nn.Linear(42, 256)
    self.hidden2 = nn.Linear(256, 128)
    self.output = nn.Linear(128, 28)
  
  def forward(self, x):
    x = self.hidden1(x)
    x = F.relu(x)
    x = self.hidden2(x)
    x = F.relu(x)
    x = self.output(x)
    return x

In [11]:
model = Model()

In [12]:
# model.load_state_dict(torch.load('landmark_model_100.pth'))

In [13]:
# magic function to load tensorboard
# %load_ext tensorboard

In [14]:
# load tensorboard
# %tensorboard --logdir tensor_log

In [15]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay= 1e-6, momentum = 0.9, nesterov = True)

In [16]:
epochs = 1000

# train_logger = tb.SummaryWriter(path.join("tensor_log", 'train_cnn'),
#                                 flush_secs=1)
# valid_logger = tb.SummaryWriter(path.join("tensor_log", 'valid_cnn'),
#                                 flush_secs=1)

# global_step_train = 0

for epoch in range(1, epochs+1): ## run the model for 100 epochs
    train_loss, valid_loss = [], []
    train_confusionMatrix = ConfusionMatrix()
    valid_confusionMatrix = ConfusionMatrix()
    ## training part 
    model.train()

    for _, batch in enumerate(trainloader):
        
        data, target = batch['data'], batch['target']

        optimizer.zero_grad()

        ## 1. forward propagation
        output = model(data)

        train_confusionMatrix.add(output.argmax(1), target.squeeze(1))

        ## 2. loss calculation
        loss = loss_function(output, target.squeeze(1))

        ## 3. backward propagation
        loss.backward()

        ## 4. weight optimization
        optimizer.step()

        # global_step_train += 1
        # train_logger.add_scalar("loss", loss,
        #                         global_step=global_step_train)
                    
        train_loss.append(loss.item())
        
    ## evaluation part 
    model.eval()

    for _, batch in enumerate(validloader):

        data, target = batch['data'], batch['target']
        output = model(data)
        valid_confusionMatrix.add(output.argmax(1), target.squeeze(1))
        loss = loss_function(output, target.squeeze(1))
        valid_loss.append(loss.item())

    # valid_logger.add_scalar("accuracy",
    #                         np.mean(valid_loss),
    #                         global_step=global_step_train)
    
    # train_logger.add_scalar('lr', optimizer.param_groups[0]['lr'],
    #                         global_step=global_step_train)
    
    print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss),
           "Training Acc: ", train_confusionMatrix.global_accuracy, 
           "Valid Loss: ", np.mean(valid_loss),
           "Valid Acc: ", valid_confusionMatrix.global_accuracy,
           )
    
    if epoch%100 == 0 and epoch != 0:
        torch.save(model.state_dict(), f'landmark_model_{epoch}.pth')

#torch.save(model.state_dict(), f'landmark_model_{epochs}.pth')

Epoch: 1 Training Loss:  3.2394567756091845 Training Acc:  tensor(0.7722) Valid Loss:  3.0488153945567995 Valid Acc:  tensor(0.4291)
Epoch: 2 Training Loss:  2.628077774889329 Training Acc:  tensor(0.5347) Valid Loss:  2.2137986449308173 Valid Acc:  tensor(0.8441)
Epoch: 3 Training Loss:  1.874100051206701 Training Acc:  tensor(0.8515) Valid Loss:  1.5453391934550085 Valid Acc:  tensor(0.9035)
Epoch: 4 Training Loss:  1.3620731984867769 Training Acc:  tensor(0.9211) Valid Loss:  1.1944670067277066 Valid Acc:  tensor(0.9584)
Epoch: 5 Training Loss:  1.114149385690689 Training Acc:  tensor(0.9558) Valid Loss:  1.0234051194301872 Valid Acc:  tensor(0.9764)
Epoch: 6 Training Loss:  0.9732178467161515 Training Acc:  tensor(0.9707) Valid Loss:  0.9054113321526106 Valid Acc:  tensor(0.9738)
Epoch: 7 Training Loss:  0.8738311013754676 Training Acc:  tensor(0.9804) Valid Loss:  0.8263282692709635 Valid Acc:  tensor(0.9822)
Epoch: 8 Training Loss:  0.7979603567544151 Training Acc:  tensor(0.9852

In [ ]:
## dataloader for validation dataset 
dataiter = iter(validloader)
batch = dataiter.next()
data, labels = batch['data'], batch['target']
output = model(data)

_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy())

print ("Actual:", labels.numpy().reshape(-1)[:10])
print ("Predicted:", preds[:10])

print(preds[:10] == labels.numpy().reshape(-1)[:10])

Actual: [ 8 25 19  9 24 25 18 20 25  2]
Predicted: [ 8 25 19  9 24 25 18 20 25  2]
[ True  True  True  True  True  True  True  True  True  True]


In [ ]:
df = pd.read_csv('asl_alphabet_train/labels.csv')
df = df.sample(frac=1).reset_index(drop=True)       
x = df.iloc[:,4:].values
y = df['label'].values.reshape(-1,1)

In [ ]:
x[1]

array([ 0.        ,  0.        ,  0.05786228, -0.06654346,  0.08304906,
       -0.14307708,  0.03804433, -0.19017434, -0.02180761, -0.21233875,
        0.05042607, -0.25779331,  0.03899133, -0.35060331,  0.02406031,
       -0.4007701 ,  0.00922436, -0.44758731, -0.00900739, -0.23962307,
       -0.01581275, -0.34495968, -0.01314265, -0.39164561, -0.00930548,
       -0.42453092, -0.06065243, -0.19943136, -0.0396719 , -0.21733195,
       -0.01066726, -0.16074717, -0.00620383, -0.14255244, -0.1006099 ,
       -0.14959776, -0.0787648 , -0.17250925, -0.04910809, -0.1371786 ,
       -0.04196107, -0.11914754])

In [ ]:
output = model(torch.tensor(x[1]).type(torch.FloatTensor))
output 

tensor([ -7.0216,   6.0504, -12.7404,   8.0520,  -5.4963, -13.0640,   2.1838,
          8.2303,   4.5476,   2.6291,   8.7274,  -3.2178,  -0.3508,   9.9614,
        -13.8822,  -2.9866, -18.4252,  16.3872,   4.8337, -10.5987,  18.4459,
         10.7970,   8.6792,   5.0900, -14.3952, -13.2475,  -0.6711,  -3.9275],
       grad_fn=<AddBackward0>)

In [ ]:
np.argmax(output.detach().numpy())

20

In [ ]:
y[1]

array([20])